In [2]:
conda install -c conda-forge pynini

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/datalore-user

  added / updated specs:
    - pynini


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openfst-1.7.5              |       he513fc3_1         7.5 MB  conda-forge
    pynini-2.0.9.post2         |   py37hc9558a2_0         1.9 MB  conda-forge
    --------------------------------

In [3]:
import pynini

In [6]:
word = 'ⲥⲟⲃⲧ Ⲍⲁⲭⲁⲣⲓⲁⲥ ⲙⲉⲧⲁⲛⲟⲓⲁ ⲃⲁⲡⲧⲓⲥⲙⲁ ⲁⲃⲣⲁⲁⲙ ϩⲟⲃⲥ ⲟⲩⲏⲃ ⲭⲉⲣⲟⲃⲓⲙ ⲉⲗⲉⲏⲥⲟⲛ.'

#TODO: Finish this loop to take in a file and iterate the above rules over its content
# with open ('filename') as f:
#     for line in f:
#         line = line.strip()
#         line @

In [7]:
#sigma

coptic_sigma = pynini.union("ⲁ", "Ⲁ", "ⲃ", "Ⲃ", "ⲅ", "Ⲅ", "ⲇ", "Ⲇ", "ⲉ", "Ⲉ", "ⲋ", "Ⲋ", "ⲍ", "Ⲍ", "ⲏ", "Ⲏ", 
                            "ⲑ", "Ⲑ", "ⲓ", "Ⲓ", "ⲕ", "Ⲕ", "ⲗ", "Ⲗ", "ⲙ", "Ⲙ", "ⲛ", "Ⲛ", "ⲝ", "Ⲝ", "ⲟ", "Ⲟ", 
                            "ⲡ", "Ⲡ", "ⲣ", "Ⲣ", "ⲥ", "Ⲥ", "ⲧ", "Ⲧ", "ⲩ", "Ⲩ", "ⲫ", "Ⲫ", "ⲭ", "Ⲭ", "ⲯ", "Ⲯ", 
                            "ⲱ", "Ⲱ", "ϣ", "Ϣ", "ϥ", "Ϥ", "ϧ", "Ϧ", "ϩ", "Ϩ", "ϫ", "Ϫ", "ϭ", "Ϭ", "ϯ", "Ϯ")

latin_sigma = pynini.union("a", "A", "b", "B", "c", "C", "d", "D", "e", "E", "f", "F", "g", "G", "h", "H", "i", 
                           "I", "j", "J", "k", "K", "l", "L", "m", "M", "n", "N", "o", "O", "p", "P", "q", "Q", 
                           "r", "R", "s", "S", "t", "T", "u", "U", "v", "V", "w", "W", "x", "X", "y", "Y", "z", 
                           "Z")

#missing backslash "\" due to "FstStringCompilationError: String compilation failed" error
punct_whitespace_sigma = pynini.union("!", '"', "#", "$", "%", "&", "'", "(", ")", "*", "+", ",", "-",
                         ".", "/", ":", ";", "<", "=", ">", "?", "@", "\[", "\]", "^", "_", 
                          "`", "{", "|", "}", "~", " ")

wb = "[WB]"

ipa_sigma = pynini.union("æ", "ə", "ɛ", "ɑː")

vowels = pynini.union("ⲁ", "Ⲁ", "ⲟ", "Ⲟ", "ⲱ", "Ⲱ", "ⲓ", "Ⲓ", "ⲏ", "Ⲏ", "ⲉ", "Ⲉ")

sigma = pynini.union(coptic_sigma, latin_sigma, punct_whitespace_sigma, vowels, ipa_sigma, wb)

sigma_star = pynini.closure(sigma)

In [8]:
#rules

insert_wb = pynini.transducer("", "[WB]")
# pynini.t("", "[WB]") + sigma_star + pynini.t("", "[WB]")
# Add WB when coptic letters are on the left and whitespace or punctuation are on the right
rule_addwb_1 = pynini.cdrewrite(insert_wb, coptic_sigma, punct_whitespace_sigma, sigma_star)
# Add WB when whitespace or punctuation are on the left and coptic letters are on the right
rule_addwb_2 = pynini.cdrewrite(insert_wb, punct_whitespace_sigma, coptic_sigma, sigma_star)

rule_removewb = pynini.cdrewrite(pynini.t("[WB]", ""), "", "", sigma_star)

#alpha

alphatoa_1 = pynini.transducer("ⲁ", "ɑː")
###rule_1###

alphatoa_2 = pynini.transducer("ⲁ", "æ")
rule_2 = pynini.cdrewrite(alphatoa_2, "", "ⲥ[WB]", sigma_star)

alphatoa_3 = pynini.transducer("ⲁ", "ɛ")
###rule_2###

alphatoa_4 = pynini.transducer("ⲁ", "ə")
rule_3 = pynini.cdrewrite(alphatoa_4, "", wb, sigma_star)

#veeta

veetatov = pynini.transducer("ⲃ", "v")
rule_5 = pynini.cdrewrite(veetatov, "", "ⲧ[WB]", sigma_star)
rule_6 = pynini.cdrewrite(veetatov, "", vowels, sigma_star)

veetatob = pynini.transducer("ⲃ", "b")
rule_7 = pynini.cdrewrite(veetatob, "", "ⲣ", sigma_star)
rule_8 = pynini.cdrewrite(veetatob, "", "ⲥ", sigma_star)
rule_9 = pynini.cdrewrite(veetatob, "", wb, sigma_star)
#this rule should be executed first due to the overlap in the vowel "ⲓ"
rule_4 = pynini.cdrewrite(veetatob, "", "ⲓⲙ[WB]", sigma_star)

#Gamma

gammatog = pynini.transducer("ⲅ", "g")
rule_11 = pynini.cdrewrite(gammatog, "", "ⲓ", sigma_star)
rule_12 = pynini.cdrewrite(gammatog, "", "ⲉ", sigma_star)

gammaton = pynini.transducer("ⲅ", "n")
#this rule should be executed first
rule_10 = pynini.cdrewrite(gammaton, "", "ⲅ", sigma_star)

gammatogh = pynini.transducer("ⲅ", "gh")
rule_13 = pynini.cdrewrite(gammatogh, "", "", sigma_star)

#delta

deltatoth = pynini.transducer("ⲇ", "th")
###rule_14###

deltatod = pynini.transducer("ⲇ", "d")
###rule_15###

# sooto6 = pynini.transducer("ⲋ", "6")
# rule_16 = pynini.cdrewrite(sooto6, "", sigma_star)

#ei
eitoe_1 = pynini.transducer("ⲉ", "eɪ")
rule_18 = pynini.cdrewrite(eitoe_1, "", "ⲟ", sigma_star)

eitoe_2 = pynini.transducer("ⲉ", "e")
rule_19 = pynini.cdrewrite(eitoe_2, "", "", sigma_star)

#eeta
eetatoy = pynini.transducer("ⲏ", "y")
#should be executed befire ei
rule_17 = pynini.cdrewrite(eetatoy, "ⲉ", "", sigma_star)

eetatoee = pynini.transducer("ⲏ", "ee")
rule_20 = pynini.cdrewrite(eetatoee, "", "", sigma_star)

TypeError: TypeError: cdrewrite() takes at least 4 positional arguments (3 given)

In [ ]:
#1:1 mapping

onetoone = pynini.string_file('/usr/local/google/home/mshehata/Desktop/coptic.txt')
rule_onetoone = pynini.cdrewrite(onetoone, "", "", sigma_star)

In [0]:
#cdrewrite

cascade = pynini.optimize(rule_addwb_1@rule_addwb_2@rule_1@rule_2@rule_3@rule_4@rule_5@rule_6@rule_7@rule_8@rule_9@rule_10@rule_11@rule_12@rule_13@rule_17@rule_18@rule_19@rule_20@rule_onetoone)

In [0]:
translit_text = (word@cascade).stringify()
print(translit_text)

In [ ]:
# TODO: Finish this loop to take in a file and iterate the above rules over its content
with open ('/usr/local/google/home/mshehata/Desktop/coptic-test.txt') as f:
    for line in f:
        final = line.strip()
        final = (line@cascade).stringify
        

print(final)

In [ ]:
#Debugging
def debug_cascade(word, *rules):
    curr = word
    print("Original:", curr)
    for i, rule in enumerate(rules):
        curr @= rule
        print(f"Stage {i+1}:", curr.stringify())

debug_cascade(word, rule_1, rule_2, rule_3, rule_4, rule_5, rule_6, rule_7, 
              rule_8, rule_9, rule_10, rule_11, rule_12, rule_13, rule_14, 
              rule_15, rule_19, rule_20, rule_21, rule_22, rule_30)